In [61]:
import glob
import os
import pandas as pd
import numpy as np
import datetime

In [70]:
# 指定したファイルパスのcsvファイルを全部読み込む関数
def read_csv_all(target_path = '../input/'):
    for file in glob.glob(target_path+'*.csv'):
        raw_name = 'raw_' + file.replace(target_path,'').replace('.csv','')
        print('read {} as {}'.format(file, raw_name))
        exec('{} = pd.read_csv(\'{}\')'.format(raw_name, file), globals())
        
# trainとtestの加工
def processing_train_test(df):
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

# メモリの節約
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# バイナリ化
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [71]:
read_csv_all()

read ../input/new_merchant_transactions.csv as raw_new_merchant_transactions
read ../input/test.csv as raw_test
read ../input/merchants.csv as raw_merchants
read ../input/historical_transactions.csv as raw_historical_transactions
read ../input/train.csv as raw_train
read ../input/sample_submission.csv as raw_sample_submission


In [72]:
raw_historical_transactions['purchase_date'] = pd.to_datetime(raw_historical_transactions['purchase_date'])
raw_new_merchant_transactions['purchase_date'] = pd.to_datetime(raw_new_merchant_transactions['purchase_date'])

In [73]:
raw_historical_transactions.dtypes

authorized_flag                 object
card_id                         object
city_id                          int64
category_1                      object
installments                     int64
category_3                      object
merchant_category_id             int64
merchant_id                     object
month_lag                        int64
purchase_amount                float64
purchase_date           datetime64[ns]
category_2                     float64
state_id                         int64
subsector_id                     int64
dtype: object

In [74]:
raw_new_merchant_transactions.dtypes

authorized_flag                 object
card_id                         object
city_id                          int64
category_1                      object
installments                     int64
category_3                      object
merchant_category_id             int64
merchant_id                     object
month_lag                        int64
purchase_amount                float64
purchase_date           datetime64[ns]
category_2                     float64
state_id                         int64
subsector_id                     int64
dtype: object

In [21]:
train = processing_train_test(raw_train)
test = processing_train_test(raw_test)

In [22]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,92


In [67]:
historical_transactions = binarize(raw_historical_transactions)
new_transactions = binarize(raw_new_merchant_transactions)

In [68]:
historical_transactions['month_diff'] = ((datetime.datetime.today() - historical_transactions['purchase_date']).dt.days)//30
historical_transactions['month_diff'] += historical_transactions['month_lag']

new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff'] += new_transactions['month_lag']

In [69]:
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=[ 'category_2', 'category_3'])

historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

# agg_fun = {'authorized_flag': ['mean']}
# auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
# auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
# auth_mean.reset_index(inplace=True)

# authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
# historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

Mem. usage decreased to 1554.77 Mb (47.2% reduction)
Mem. usage decreased to 101.09 Mb (49.1% reduction)


In [34]:
?pd.get_dummies

In [36]:
historical_transactions.head()

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,...,category_1_N,category_1_Y,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,Y,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,...,1,0,1,0,0,0,0,1,0,0
1,Y,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,...,1,0,1,0,0,0,0,1,0,0
2,Y,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,...,1,0,1,0,0,0,0,1,0,0
3,Y,C_ID_4e6213e9bc,88,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,...,1,0,1,0,0,0,0,1,0,0
4,Y,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,...,1,0,1,0,0,0,0,1,0,0


In [52]:
test1 = historical_transactions.columns
test2 = test1[2]
historical_transactions.dtypes

authorized_flag                 object
card_id                         object
city_id                          int64
installments                     int64
merchant_category_id             int64
merchant_id                     object
month_lag                        int64
purchase_amount                float64
purchase_date           datetime64[ns]
state_id                         int64
subsector_id                     int64
month_diff                       int64
category_1_N                     uint8
category_1_Y                     uint8
category_2_1.0                   uint8
category_2_2.0                   uint8
category_2_3.0                   uint8
category_2_4.0                   uint8
category_2_5.0                   uint8
category_3_A                     uint8
category_3_B                     uint8
category_3_C                     uint8
dtype: object